In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_excel('discount.xlsx')

### 1. Общая информация

In [3]:
# посмотрим на набор данных
data.sample(10)

,cid,lid,value,term,before_dicount,promocode,loyalty_level,loyalty_discount
12129,4591055,1476698,5000,25,1.00,NaN,0.0,0.02
3247,1629068,1459351,6800,10,1.00,NaN,0.0,0.02
8138,4523236,1468644,20000,42,0.99,0.15,NaN,NaN
20140,4454687,1492756,34500,168,0.99,0.26,NaN,NaN
5665,4535749,1463973,9200,56,0.99,0.15,NaN,NaN
12571,2189502,1477538,15000,30,1.00,NaN,6.0,0.12
25598,4567630,1503644,4000,30,1.00,NaN,3.0,0.06
8760,4593693,1469869,4000,30,1.00,NaN,NaN,NaN
1446,4290060,1455648,6000,30,1.00,NaN,0.0,0.02
14863,3411936,1482142,11500,30,1.00,NaN,4.0,0.08


- cid - id клиента
- lid - id займа
- value - сумма займа
- term - запрошенный срок займа
- before_dicount - ставка до скидки
- promocode - скидка по промокоду
- loyalty_level - уровень программы лояльности
- loyalty_discount - скидка по программе лояльности

In [4]:
# посмотрим на общую информацию таблицы
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cid               30000 non-null  int64  
 1   lid               30000 non-null  int64  
 2   value             30000 non-null  int64  
 3   term              30000 non-null  int64  
 4   before_dicount    30000 non-null  float64
 5   promocode         9156 non-null   float64
 6   loyalty_level     18736 non-null  float64
 7   loyalty_discount  18736 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 1.8 MB


In [5]:
# проверим таблицу на наличие дубликатов
data.duplicated().sum()

0

In [6]:
# проверим таблицу на наличие пропусков
data.isnull().sum()

cid                     0
lid                     0
value                   0
term                    0
before_dicount          0
promocode           20844
loyalty_level       11264
loyalty_discount    11264
dtype: int64

<div class="alert alert-block alert-info">
<ul>
    <li> в таблице 30 000 строк
    <li> каждая строка - информация об уникальном займе
    <li> в столбцах pc_discount, loyalty_level, ll_discount есть пропуски
</ul>
</div>

### 2. Исследование скидочных промокодов

#### 2.1 Общая информация

In [ ]:
data['promocode']